In [ ]:
def _manual_calibrate(self, f_data, z_data, ignoreslope=True,guessdelay=True,fixed_delay=None):
        """
        Finds the parameters for normalization of the scattering data. See
        Sij of port classes for explanation of parameters.
        """
        
        # Correct for delay and translate circle to origin
        self.delay, params = self.manual_fit_phase(f_data,z_data)
        z_data = self.z_data_raw * np.exp(2j*np.pi*self.delay*self.f_data)
        xc, yc, self.r0 = self._fit_circle(z_data)
        zc = complex(xc, yc)
        z_data -= zc
        
        # Find off-resonant point by fitting offset phase
        # (centered circle corresponds to lossless resonator in reflection)
        self.fr, self.Ql, theta, self.delay_remaining = self._fit_phase_x(z_data)
        self.theta = self._periodic_boundary(theta, np.pi)
        # beta = self._periodic_boundary(theta - np.pi)
        beta = self._periodic_boundary(theta+np.pi,np.pi)
        offrespoint = zc + self.r0*np.cos(beta) + 1j*self.r0*np.sin(beta)
        self.offrespoint = offrespoint
        #print(f"offresspoint = {offrespoint}")
        self.a = np.absolute(offrespoint)
        #self.alpha = np.angle(offrespoint)
        self.alpha = self._periodic_boundary(np.angle(offrespoint)+np.pi,np.pi)
        #print(f"alpha = {self.alpha}")
        self.phi = self._periodic_boundary(beta - self.alpha, np.pi)
        #print(f"phi = {self.phi}")
        print(f"man calib: r0 = {self.r0}, zc = {zc}")
        # Store radius for later calculation
        self.r0 /= self.a
        